<a href="https://colab.research.google.com/github/luqmanr/web-scraping/blob/master/instagram-crawler/instagram_crawler_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!python3 -m pip install opencv-contrib-python>=4.1.0
!pip install stdiomask
!apt install chromium-chromedriver
!pip install selenium
!pip install cp
!sudo apt-get install python-yaml
!sudo yum install python-yaml
!apt-get update
!apt install python3-dev python3-pip
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

  Created wheel for stdiomask: filename=stdiomask-0.0.5-cp36-none-any.whl size=3318 sha256=afb2ef8d7172c49b6bd251868c7e24490deb63ba644b905f9b25121664c85cf1
  Stored in directory: /root/.cache/pip/wheels/da/05/d3/b1ac160dd86577b6606330bc1d9fccf7d5dbe1713352a847df
Successfully built stdiomask
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 25 not upgraded.
Need to get 74.4 MB of archives.
After this operation, 264 MB of additional disk space will be used.
Ign:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffm

In [12]:
import getpass

login_id = input('Your IG Account Username: ')
password = getpass.getpass('Your IG Account Password: ')

Your IG Account Username: tyogotest_2
Your IG Account Password: ··········


In [0]:
usernames_target = [
  # "antoni.lovendio", "ancellastrore", "indrtmy", "aurelius.nathanael", "gxlua", "davidhendriawan_", "gigagaharu"
  "tyogo_utomo"
]

In [0]:
import os, time, errno, pickle
import re
import json
import csv
import yaml
import sys
import cp
import selenium
from selenium import webdriver
from urllib.request import urlopen
from optparse import OptionParser
from selenium import webdriver  
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

infinity = 0x40000
maxCrawl = infinity
check = False
timeout = False
follGetError = False
timeoutValue = 360
timeoutCounter = 0
crawlCounter = 0
counter = 0
scroll = 0
folllist = []
follidlist = []
filename = "crawled_username.csv"

WINDOW_SIZE = "1360,768"
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
os.chdir("/content/gdrive/My Drive/")

driver = webdriver.Chrome(
    executable_path="/usr/lib/chromium-browser/chromedriver",
    chrome_options=chrome_options
)

def login_instagram():
    try:
        driver.get("https://www.instagram.com/accounts/login/")
        time.sleep(2)
        driver.find_element_by_name("username").send_keys(login_id)
        driver.find_element_by_name("password").send_keys(password)
        time.sleep(1)
        driver.find_element_by_class_name("L3NKy").click()
        return login_id
    except:
        print("the Xpath of this element (login_instagram) has changed")
        time.sleep(3)
        driver.close

def fullName():
  followingButtonXpath = '/html/body/div[1]/section/main/div/header/section/ul/li[3]/a/span'
  following_button = driver.find_element_by_xpath(followingButtonXpath).text
  print("Total Following: " + following_button)
  postButtonXpath = '/html/body/div[1]/section/main/div/header/section/ul/li[1]/span/span'
  post_button = driver.find_element_by_xpath(postButtonXpath).text
  print("Total posts: " + post_button)
  fullNameXpath = '/html/body/div[1]/section/main/div/header/section/div[2]/h1'
  full_name = driver.find_element_by_xpath(fullNameXpath).text
  print("Full Name: " + full_name)
  return full_name, post_button, following_button

def followerButton():
    try:
        follButtonXpath = '/html/body/div[1]/section/main/div/header/section/ul/li[2]/a/span'
        follower_button = driver.find_element_by_xpath(follButtonXpath)
        printfoll = follower_button.get_attribute('title')
    except KeyError:
        follButtonXpath = '/html/body/div[1]/section/main/div/ul/li[2]/span'
        follower_button = driver.find_element_by_xpath(follButtonXpath)
        print("followerButton Xpath alt 2")
        pass
    except:
        pass
        print("the Xpath of this element (followerButton) has changed")
    follValue = printfoll.split(',')
    try:
      follvalue0 = follValue[0]
      follvalue1 = follValue[1]
      follvalue = str(follvalue0 + follvalue1)
      print("Total followers: " + follvalue)
      follower_button.click()
      return follvalue
    except:
      follvalue0 = follValue[0]
      follvalue = str(follvalue0)
      follvalue = follvalue0
      print("Total followers: " + follvalue)
      follower_button.click()
      return follvalue

def follGet(n):
  follmax = 300
  while n <= follmax:
    str_n = str(n)
    try:
      follXpath = '/html/body/div[4]/div/div[2]/ul/div/li[' + str_n + ']/div/div[1]/div[2]/div[1]/a'
    except KeyError:
      follXpath = '/html/body/div[4]/div/div[2]/ul/div/li[' + str_n + ']/div/div[2]/div[1]/div/div/a'
    except:
        pass
        print("the Xpath of this element (follGet) has changed")
    finally:
      follower_name = driver.find_element_by_xpath(follXpath)
      follvalue = (follower_name.text)
      time.sleep(0.2)
      return follvalue
    if n > follmax:
      break
    else:
      pass

def ID():
    urlmax = 300  #maximal userID to take (to avoid account block by Instagram)
    while counter <= urlmax:
        driver.switch_to.window(driver.window_handles[1])
        driver.get("https://www.instagram.com/"+USERstr+"/?__a=1")
        time.sleep(0.3)
        userID=driver.find_element_by_xpath('/html/body/pre')
        user=userID.get_attribute('innerHTML')
        s=user
        y=yaml.load(s)
        l=(y.get("logging_page_id"))
        IGid=(l.split('_'))
        IGidVal = IGid[1]
        driver.switch_to.window(driver.window_handles[0])
        if counter >= urlmax:
            print("ID Limit exceded")
            break
        return IGidVal

def myID():
    driver.switch_to.window(driver.window_handles[1])
    driver.get("https://www.instagram.com/"+login_id+"/?__a=1")
    time.sleep(0.3)
    userID=driver.find_element_by_xpath('/html/body/pre')
    user=userID.get_attribute('innerHTML')
    s=user
    y=yaml.load(s)
    l=(y.get("logging_page_id"))
    IGid=(l.split('_'))
    myIGidVal = IGid[1]
    driver.switch_to.window(driver.window_handles[0])
    return myIGidVal

################################################################################
os.chdir("/content/gdrive/My Drive/")
try:
    os.mkdir("./instagram-crawling")
except:
    pass
os.chdir("/content/gdrive/My Drive/instagram-crawling")
try:
    os.mkdir("./instagram-usernames")
except:
    pass
os.chdir("/content/gdrive/My Drive/instagram-crawling/instagram-usernames")
try:
  open('crawled_username.csv', 'r')
  pass
except:
  with open('crawled_username.csv', 'w', newline='') as csvfile:
    wr = csv.writer(csvfile)

login_instagram()
time.sleep(5)
usernames_total = len(usernames_target)
print("total of usernames to crawl: " + str(usernames_total))
if usernames_total == 0:
  sys.stderr.write("There is no username to crawl. Please enter at least 1 username to crawl! \n")
  driver.quit()
  exit
else:
  pass
while crawlCounter < 10: #usernames_total: #
  user_target = usernames_target[crawlCounter]
  with open(filename) as csvread:
    reader = csv.reader(csvread)
    for row in reader:
      if user_target in row:
        print("This username is already crawled")
        crawlCounter += 1
        check = True
      else:
        check = False
  if check == True:
    continue
  elif check == False:
    pass
  
  driver.execute_script("window.open('');")
  driver.switch_to.window(driver.window_handles[0])
  driver.get('https://www.instagram.com/'+ user_target + '/')
  time.sleep(5)
  
  crawlCounter += 1
  timeoutCounter += 1
  if timeoutCounter == 2:
    timeoutCounter = 0
    timeout = True
    print("timeout engage")
  else:
    timeout = False
    pass

  try:
    follower_button=followerButton()
  except:
    sys.stderr.write("This account is private or doesn't exist \n")
    continue
  
  myIDstr = str(myID())
  print(myIDstr)
  follower_value = int(follower_button)
  full_name, post_button, following_button = fullName()
  time.sleep(3)

  #scrolling
  fBody = driver.find_element_by_xpath("//div[@class='isgrP']")
  while scroll < follower_value:
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;', fBody)
    time.sleep(0.1)
    scroll += 1
    # if scroll == follower_value/2:
    if scroll == 120:
      print("scrolling finish")
      break
    else:
      pass
  time.sleep(1)
  
  for i in range(1, follower_value):
    try:
      USERstr = str(follGet(i))
      follGetError = False
    except:
      sys.stderr.write("error occured")
      follGetError = True
      break
    print(str(i)+USERstr)
    if USERstr == "None":
      break
    else:
      pass
    IDstr = str(ID())
    print(IDstr)
    if IDstr == "None":
        IDstr = "ID limit exceded"
    else:
        pass
    usernames_target.append(USERstr)
    folllist.append(USERstr)
    follidlist.append(IDstr)

  if follGetError == False:
    print(folllist)
    folllistRow1 = folllist[0]
    folllist = folllist[1:follower_value]
  else:
    continue
    
  with open('crawled_username.csv', 'a+', newline='') as csvwrite:
    wr = csv.writer(csvwrite)
    wr.writerow([user_target])

  with open('followers(' + user_target + ').csv', 'w', newline='') as csvfile:
      wr = csv.writer(csvfile)
      wr.writerow(["Root"] + ["Username"] + ["Username ID"] + ["Full Name"] + ["Total Posts"] + ["Total Followers"] + ["Total Following"] + ["Followers"] + ["Followers ID"])
      wr.writerow([login_id] + [user_target] + [myIDstr] + [full_name] + [post_button] + [follower_button] + [following_button] + [folllistRow1])

      for follower in folllist:
        wr.writerow([""]+[""]+[""]+[""]+[""]+[""]+[""]+[follower])# + [follidlist])
  
  driver.execute_script("window.open('');")
  try:
    driver.switch_to.window(driver.window_handles[1])
    driver.execute_script("window.close('');")
  except:
    pass
  try:
    driver.switch_to.window(driver.window_handles[0])
    driver.execute_script("window.close('');")
  except:
    pass
  folllist=[]
  time.sleep(timeout)
'''  if timeout == True:
    print("timeout!")
    time.sleep(timeoutValue)
  else:
    time.sleep(3)
'''

usernames_target=[]
time.sleep(5)
driver.quit()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: use options instead of chrome_options


total of usernames to crawl: 1
Total followers: 496
33747889699
Total Following: 597
Total posts: 59
Full Name: Dionisius Tyogo Djatmiko Utomo
scrolling finish
1mrbenmeister
48677643
2math_test_preparation
32474966222
3aurelius.nathanael
15990806875
4tiusevan
32023081328
5indrtmy
2054675254
6k.f.ranu_170307
23123510891
7chris_tofer94paulus
33367926299
8m.p.f.l_23
15760300110
9timothy.s.y
1912620865
10jooooosalim
32222585928
11pinjamsignal
4307912057
12markitongs
13299882749
13teodorusjordanp
4305424482
14kuaisin
11103303514
15tyogotest
31741907053
16paulsoareshutama_94
31502669850
17sofashabby01
6932660053
18nathchriist
4102659133
19surabi052
4345943669
201eejhonson
1465078589
21iscaamanda
1379965633
22idiotic_kidots
8105308655
23luqmanr
33214295
24hollanastax
17330549064
25inilahniko
54008595
26agile_lordax
14532799280
27coconut.brah
30045252266
28debukocheng
30073432981
29dr._doni
29416623155
30m.rizkyrani
4834323174
31yourkomrade_
28417532547
32johnsonmark1821
9212536315
33dita.irja